In [26]:
# Copyright 2022 The Forta Foundation
 
import requests
import pandas as pd
import json
import dotenv
import os

# Load environment variables
dotenv.load_dotenv()

True

In [27]:
def execute_query(sql: str) -> pd.DataFrame:
        url = "https://q.luabase.com/run"
        payload = {"block":
                   {"details": {
                    "sql": sql,
                    }
                    },
                   "api_key": os.environ.get('LUABASE_API_KEY')
                   }

        headers = {"content-type": "application/json"}
        response = requests.request("POST", url, json=payload, headers=headers, timeout=600)  # 10min timeout
        
        data = response.json()
        
        print(data)
        return pd.DataFrame(data["data"])

This notebook allows to backtest the attack detector V2 against a set of alerts. Independent of the threshold set within the bot, the log will be analyzed, so the relevant alerts, their anomaly score, overall anomaly score, and any FP filtering mitigations are shown.

Since there is a command line character limit, the local persistence mechanism needs to be enabled to persist state across executions (by setting local node property in constants.py to 1)

In [28]:
start_date = "2022-12-31" #when the attack detector first started to operate

# attacker_addresses = "0x14d8Ada7A0BA91f59Dc0Cb97C8F44F1d177c2195".lower()
# chain = "ethereum" #ethereum or binance 
# name = "TomInu"
# end_date = "2023-01-26" #should be the date of the attack


# attacker_addresses = "0x69f0EdC352eCffC4EF49516c9b20eA88B3E947cb".lower()
# chain = "ethereum" #ethereum or binance 
# name = "BlueCluesInu"
# end_date = "2023-01-26" #should be the date of the attack

attacker_addresses = "0x88a2386e7ec97ad1e7a72176a66b6d0711ae3527".lower()
chain = "ethereum" #ethereum or binance 
name = "Quaternion"
end_date = "2023-01-20" #should be the date of the attack


# attacker_addresses = "0xf3a465c9fa6663ff50794c698f600faa4b05c777,0x2ffc59d32a524611bb891cab759112a51f9e33c0" #comma separated list of attacker addresses (note, while all alerts from those addresses are pushed to the attack detector, one should check the clustering bot to see whether they are clustered together)
# chain = "ethereum" #ethereum or binance 
# name = "ANKR"
# end_date = "2022-12-04" #should be the date of the attack

# start_date = "2023-01-10" #when the attack detector first started to operate
# attacker_addresses = "0x9d0163e76bbcf776001e639d65f573949a53ab03"
# chain = "ethereum" #ethereum or binance 
# name = "LendHub"
# end_date = "2023-01-14" #should be the date of the attack

# attacker_addresses = "0x0348d20b74ddc0ac9bfc3626e06d30bb6fac213b"
# chain = "ethereum" #ethereum or binance 
# name = "JayExploiter"
# end_date = "2022-12-31" #should be the date of the attack

# attacker_addresses = "0x001B91c794dFEecf00124D3F9525DD32870B6ee9" #comma separated list of attacker addresses (note, while all alerts from those addresses are pushed to the attack detector, one should check the clustering bot to see whether they are clustered together)
# chain = "ethereum" #ethereum or binance 
# name = "CryptoRubric"
# end_date = "2022-12-28" #should be the date of the attack

# attacker_addresses = "0x3bdF01ed32F07e8e843163b5d478d4502F5743CD,0x25fDe76A52D01c83E31d2d3D5e1d2011ff103c56,0xBeAdeDBABED6A353c9cAa4894Aa7E5F883e32967" #comma separated list of attacker addresses (note, while all alerts from those addresses are pushed to the attack detector, one should check the clustering bot to see whether they are clustered together)
# chain = "ethereum" #ethereum or binance 
# name = "ElasticSwap"
# start_date = "2022-11-30" #when the attack detector first started to operate
# end_date = "2022-12-15" #should be the date of the attack

# attacker_addresses = "0x86Aa1c46f2Ae35ba1B228dC69fB726813D95b597,0x3c4e5b099f3c02122079d124138377e1b9048629" #comma separated list of attacker addresses (note, while all alerts from those addresses are pushed to the attack detector, one should check the clustering bot to see whether they are clustered together)
# chain = "binance" #ethereum or binance 
# name = "Nimbus"
# end_date = "2022-12-15" #should be the date of the attack

# attacker_addresses = "0xF4FD2EbE7196c8E99E88bcc4Aef69dda0e493B8F" #comma separated list of attacker addresses (note, while all alerts from those addresses are pushed to the attack detector, one should check the clustering bot to see whether they are clustered together)
# chain = "binance" #ethereum or binance 
# name = "BlackGold"
# end_date = "2022-12-15" #should be the date of the attack

# attacker_addresses = "0x286E09932B8D096cbA3423d12965042736b8F850" #comma separated list of attacker addresses (note, while all alerts from those addresses are pushed to the attack detector, one should check the clustering bot to see whether they are clustered together)
# chain = "binance" #ethereum or binance 
# name = "aespool"
# end_date = "2022-12-17" #should be the date of the attack

# attacker_addresses = "0xf71708C59be7e32B9FF2aa174F07311869C6bf0c" #comma separated list of attacker addresses (note, while all alerts from those addresses are pushed to the attack detector, one should check the clustering bot to see whether they are clustered together)
# chain = "binance" #ethereum or binance 
# name = "optionroom"
# end_date = "2022-12-17" #should be the date of the attack




In [29]:
attacks = [("0x14d8Ada7A0BA91f59Dc0Cb97C8F44F1d177c2195","ethereum","tominu","2023-01-18","2023-01-27"),
           ("0x69f0EdC352eCffC4EF49516c9b20eA88B3E947cb","ethereum","blueclues","2023-01-19","2023-01-26"),
           ("0x88a2386e7ec97ad1e7a72176a66b6d0711ae3527","ethereum","quaternion","2023-01-17","2023-01-19"),
           ("0x8a2d94ea342cbdd6d57db614b24f20cae286cac6,0x22898dc59145eae79471dd1f06b7e542e0982d30","ethereum","Upswing","2022-12-21","2023-01-20"),
           ("0x15d87DC2Eb27fdA26451f8FB04C576639104344d","ethereum","roefinance","2023-01-10","2023-01-18"),
           ("0x9D0163e76BbCf776001E639d65F573949a53AB03","ethereum","lendhub","2023-01-10","2023-01-16")
           ]

# attacks = [("0x66be80c796cba0844dace3e291632bfd397bd7a0","binance","phyproxy","2023-01-01","2023-01-24"),
#            ("0x1ae2dc57399b2f4597366c5bf4fe39859c006f99","binance","thoreum","2023-01-14","2023-01-20"),
#            ("0x9BbD94506398a1459F0Cd3B2638512627390255e","binance","omniprotocol","2023-01-16","2023-01-18"),
#            ("0xc578d755cd56255d3ff6e92e1b6371ba945e3984","binance","ufdaoattack","2023-01-01","2023-02-06"),
#            ("0x67a909f2953fb1138beA4B60894B51291D2d0795,0xE2Ba15be8C6Fb0d7C1F7bEA9106eb8232248FB8B","binance","bratoken","2022-12-08","2023-01-16"),
#            ("0xcf2362b46669e04b16d0780cf9b6e61c82de36a7","binance","GDS","2023-01-01","2023-01-05")
#             ]

In [30]:
# #clustering
# for attacker_address in attacker_addresses.split(","):
#     sql = f"SELECT metadata FROM forta.{chain}_alerts WHERE bot_id = '0xd3061db4662d5b3406b52b20f34234e462d2c275b99414d76dc644e2486be3e9' and CAST(substring(block_timestamp,1,19) as datetime)  >= '2022-10-01T00:00:00' AND CAST(substring(block_timestamp,1,19)  as datetime)  <= '{end_date}T00:00:00' and  lower(arrayStringConcat(metadata)) like '%{attacker_address.lower()}%'"
#     print(sql)
#     df = execute_query(sql)
#     print(df)

In [31]:
def get_alert_hashes(attacker_addresses: str, start_date: str, end_date: str) -> pd.DataFrame:

    alert_hashes_df = pd.DataFrame(columns=["hash","createdAt"])
    for attacker_address in attacker_addresses.split(","):
        url = 'https://api.forta.network/graphql'
        query = '''query exampleQuery {
            # first 5 alerts
            alerts(
                input: { first: 2000 , addresses: ["'''+attacker_address+'''"],
                blockDateRange: { startDate: "'''+start_date+'''", endDate: "'''+end_date+'''" }
                }
            ) {
                pageInfo {
                hasNextPage
                endCursor {
                    alertId
                    blockNumber
                }
                }
                alerts {
                hash
                createdAt
                
                }
            }
            }
        '''
        r = requests.post(url, json={'query': query})
        #print(r.text)
        json_data = json.loads(r.text)
        df = pd.DataFrame(json_data['data']['alerts']['alerts'])
        #print(df)
        alert_hashes_df = pd.concat([alert_hashes_df, df])

    print(len(alert_hashes_df))
    return alert_hashes_df


In [32]:

for attacker_addresses, chain, name, start_date, end_date in attacks:

    alert_hashes_df = get_alert_hashes(attacker_addresses.lower(), start_date, end_date)

    !echo 'output.txt' > {name}.txt

    #run npm run to get and route to output.txt
    count = 0
    alert_hashes = ""
    for alert_hash in alert_hashes_df["hash"]:
        count += 1
        alert_hashes += f",{alert_hash}"
        if count % 90 == 0:
            !npm run alert {alert_hashes} >> {name}.txt 2>> {name}.txt
            alert_hashes = ""
    if count % 90 != 0:
        !npm run alert {alert_hashes} >> {name}.txt 2>> {name}.txt


    !grep 'ERROR' {name}.txt > {name}_results.txt
    !grep 'WARN' {name}.txt >> {name}_results.txt
    !grep 'adding cluster mapping' {name}.txt >> {name}_results.txt
    !grep 'adding FP mitigation cluster' {name}.txt >> {name}_results.txt
    !grep 'anomaly' {name}.txt >> {name}_results.txt
    



27
750
214
85
17
177


In [33]:
!grep 'ERROR' {name}.txt


2023-02-06 17:32:28,998 - root - ERROR - Exception in get_etherscan_label 'NoneType' object is not subscriptable
2023-02-06 17:32:30,278 - root - ERROR - Exception in get_etherscan_label 'NoneType' object is not subscriptable
2023-02-06 17:32:32,274 - root - ERROR - Exception in get_etherscan_label 'NoneType' object is not subscriptable
2023-02-06 17:32:33,053 - root - ERROR - Exception in get_etherscan_label 'NoneType' object is not subscriptable
2023-02-06 17:32:34,804 - root - ERROR - Exception in get_etherscan_label 'NoneType' object is not subscriptable
2023-02-06 17:32:35,732 - root - ERROR - Exception in get_etherscan_label 'NoneType' object is not subscriptable
2023-02-06 17:32:36,489 - root - ERROR - Exception in get_etherscan_label 'NoneType' object is not subscriptable
2023-02-06 17:32:38,330 - root - ERROR - Exception in get_etherscan_label 'NoneType' object is not subscriptable
2023-02-06 17:32:41,617 - root - ERROR - Exception in get_etherscan_label 'NoneType' object is n

In [34]:
!grep 'WARN' {name}.txt

2023-02-06 17:32:26,869 - root - WARNING - alert 0xbe54809f879348f102fa804ad2322c647b32f22d23e2caf1a3dcd0135373f6d1 POSSIBLE-MONEY-LAUNDERING-TORNADO-CASH - no anomaly_score in metadata found: {'totalFundsTransferred': '1000'}. Treating as anomaly_score of 1.0.
2023-02-06 17:32:28,797 - root - WARNING - alert 0x1ac04eee304d0b0f5f821d0d1bfdcceb516d3552ccb309b58b1179aff3f6296a FLD_Laundering - no anomaly_score in metadata found: {'launderingAddress': '0x9d0163e76bbcf776001e639d65f573949a53ab03', 'newlyCreated': 'false', 'targetAddress': '0x4585fe77225b41b697c938b018e2ac67ac5a20c0', 'targetType': 'dex', 'token': 'WBTC', 'txHash': '0xf16a350d6f53a57496c1e63bd64650a4a3659ee9a557c388b7cf47e12c687676', 'usdVolume': '57454.0621378'}. Treating as anomaly_score of 1.0.
2023-02-06 17:32:29,784 - root - WARNING - alert 0x1117cdf56e991df9ff3531ef78342fc2ae88a4df98448c902995ea420da39db1 NETHFORTA-25 - no anomaly_score in metadata found: {'address': '0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45', 'reen

In [35]:
!grep 'adding cluster mapping' {name}.txt

In [36]:
!grep 'adding FP mitigation cluster' {name}.txt

2023-02-06 17:35:54,880 - root - INFO - alert 0xa88eb3f1c4b3e4dd755cb4a8491f3f57aaeda04d0d5cab53cfa06d90ae48f580 adding FP mitigation cluster: 0x9D0163e76BbCf776001E639d65F573949a53AB03. FP mitigation clusters size now: 1


In [37]:
#read output.txt and answer questions around
!grep 'anomaly' {name}.txt


#anomaly scores
#FP filtering


2023-02-06 17:32:26,869 - root - WARNING - alert 0xbe54809f879348f102fa804ad2322c647b32f22d23e2caf1a3dcd0135373f6d1 POSSIBLE-MONEY-LAUNDERING-TORNADO-CASH - no anomaly_score in metadata found: {'totalFundsTransferred': '1000'}. Treating as anomaly_score of 1.0.
2023-02-06 17:32:26,869 - root - INFO - alert 0xbe54809f879348f102fa804ad2322c647b32f22d23e2caf1a3dcd0135373f6d1 0x4adff9a0ed29396d51ef3b16297070347aab25575f04a4e2bd62ec43ca4508d2 POSSIBLE-MONEY-LAUNDERING-TORNADO-CASH MoneyLaundering: 0x9d0163e76bbcf776001e639d65f573949a53ab03 anomaly score of 1.0
2023-02-06 17:32:28,312 - root - INFO - alert 0xb088652a83a83ba819e208aaafda60e62741548026ddc3d1279aed63fb15e05b 0x2e51c6a89c2dccc16a813bb0c3bf3bbfe94414b6a0ea3fc650ad2a59e148f3c8 ANOMALOUS-TOKEN-TRANSFERS-TX Exploitation: 0x9d0163e76bbcf776001e639d65f573949a53ab03 anomaly score of 0.00089
2023-02-06 17:32:28,797 - root - WARNING - alert 0x1ac04eee304d0b0f5f821d0d1bfdcceb516d3552ccb309b58b1179aff3f6296a FLD_Laundering - no anomaly_sco